In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("cloudnine")

create_database('blp_dataset')
create_table('products_tmp_cloudnine')

In [ ]:
images = []
regex = re.compile(".*?\([.*?]\)")

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            
            if "_product_id" in file:
                with open(Path(f"{folder}\{file}")) as file_c:
                    try:
                        file_c = file_c.read()
                        page_content = BeautifulSoup(file_c)

                        if page_content is not None:
                            ## GET DESCRIPTION
                            description = ""
                            desc_content = page_content.find("div", attrs={"style": "padding-right: 320px;"})

                            if desc_content is not None:
                                description = get_content(desc_content)

                            ## GET TITLE
                            title = ""
                            title_content = page_content.find("h3")

                            if title_content is not None:
                                title_content.find('a')

                                if title_content is not None:
                                    title_content = title_content.decompose()
                                    title_to_clear = get_content(title_content).replace(" - ", "")

                                    title = re.sub(r'\[.*\]', '', title_to_clear)

                                if description != "" or title != "":
                                    # Insert product into database
                                    query = "INSERT INTO products_tmp_cloudnine (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
                                    values = (title, description, "Cloudnine", "t")

                                    cursor.execute(query, values)
                                    conn.commit()

                                    product_id = cursor.lastrowid

                                    # GET IMAGE
                                    raw_name = page_content.find("center")

                                    if raw_name is not None:
                                        raw_image = raw_name.find("a")

                                        if raw_image is not None:
                                            img_name = raw_image["href"]
                                            url_image = Path(f"{folder_image}\{img_name}")

                                            if url_image.exists():
                                                print(f"Image {product_id} exists... creating")

                                                dst_dir = Path(f"D:\images\_cloudnine\{product_id}.jpg")
                                                copyfile(url_image, dst_dir)

                                            else:
                                                print(f"Image {product_id} does not exists")
                    except:
                        print(f"Error {file}")